[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/tensorflow/blob/main/Part%203/3.5_callback.ipynb)

In [1]:
'''
import tensorflow as tf

# 케라스의 내장 데이터셋에서 mnist 데이터셋을 로드
mnist = tf.keras.datasets.mnist

# load_data()로 데이터셋을 로드 합니다.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 로드된 데이터셋 확인
print('train set: ', x_train.shape, y_train.shape)
print('test  set: ', x_test.shape, y_test.shape)

# 데이터 정규화
x_train = x_train / x_train.max() # max: 255
x_test = x_test / x_test.max() # max: 255
'''

"\nimport tensorflow as tf\n\n# 케라스의 내장 데이터셋에서 mnist 데이터셋을 로드\nmnist = tf.keras.datasets.mnist\n\n# load_data()로 데이터셋을 로드 합니다.\n(x_train, y_train), (x_test, y_test) = mnist.load_data()\n\n# 로드된 데이터셋 확인\nprint('train set: ', x_train.shape, y_train.shape)\nprint('test  set: ', x_test.shape, y_test.shape)\n\n# 데이터 정규화\nx_train = x_train / x_train.max() # max: 255\nx_test = x_test / x_test.max() # max: 255\n"

In [1]:
import numpy as np
import tensorflow as tf

# 파일 경로 설정 (환경에 맞게 수정하세요)
mnist_path = '../dataset/mnist/mnist.npz'  # 윈도우 예시
#mnist_path = 'D:/datasets/mnist.npz'  # 윈도우 예시
# mnist_path = '/home/user/datasets/mnist.npz'  # 리눅스 예시

# npz 파일 직접 로드
with np.load(mnist_path) as data:
    x_train = data['x_train']
    y_train = data['y_train']
    x_test = data['x_test']
    y_test = data['y_test']

# 데이터 확인
print('train set: ', x_train.shape, y_train.shape)
print('test  set: ', x_test.shape, y_test.shape)

# 정규화
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

print("x_train[0, 10:15, 10:15] = " + repr(x_train[0, 10:15, 10:15]))
print("x_test[0, 10:15, 10:15] = " + repr(x_test[0, 10:15, 10:15]))

train set:  (60000, 28, 28) (60000,)
test  set:  (10000, 28, 28) (10000,)
x_train[0, 10:15, 10:15] = array([[0.00392157, 0.60392157, 0.99215686, 0.35294118, 0.        ],
       [0.        , 0.54509804, 0.99215686, 0.74509804, 0.00784314],
       [0.        , 0.04313725, 0.74509804, 0.99215686, 0.2745098 ],
       [0.        , 0.        , 0.1372549 , 0.94509804, 0.88235294],
       [0.        , 0.        , 0.        , 0.31764706, 0.94117647]])
x_test[0, 10:15, 10:15] = array([[0.        , 0.06666667, 0.25882353, 0.05490196, 0.2627451 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])


## 5-1. 모델 체크포인트

In [2]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), 
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(32, activation='relu'), 
    tf.keras.layers.Dense(10, activation='softmax'), # 노드 10개로 생성
])

c:\Python\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [3]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [4]:
# 체크포인트 설정
checkpoint = tf.keras.callbacks.ModelCheckpoint(#filepath='tmp_checkpoint.ckpt', 
                                                filepath='tmp_checkpoint.weights.h5',  # 확장자 수정
                                                save_weights_only=True, 
                                                save_best_only=True, 
                                                monitor='val_loss', 
                                                verbose=1)


In [5]:
model.fit(x_train, y_train, 
          validation_data=(x_test, y_test), 
          epochs=10, 
          callbacks=[checkpoint]
          )

Epoch 1/10
1868/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8736 - loss: 0.4137
Epoch 1: val_loss improved from None to 0.12250, saving model to tmp_checkpoint.weights.h5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9303 - loss: 0.2318 - val_accuracy: 0.9628 - val_loss: 0.1225
Epoch 2/10
1869/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9698 - loss: 0.1001
Epoch 2: val_loss improved from 0.12250 to 0.09754, saving model to tmp_checkpoint.weights.h5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9707 - loss: 0.0967 - val_accuracy: 0.9693 - val_loss: 0.0975
Epoch 3/10
1862/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9801 - loss: 0.0632
Epoch 3: val_loss improved from 0.09754 to 0.07798, saving model to tmp_checkpoint.weights.h5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9798 - loss: 0.0656 - val_accuracy: 0.9756 - val_loss: 0.0780
Epoch 4/10
1854/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9862 - loss: 0.0439
Epoch 4: val

In [6]:
# 모델 체크포인트 로드 전
loss, acc = model.evaluate(x_test, y_test)
print(f'체크포인트 로드 전: loss: {loss:.4f}, acc: {acc:.4f}')

# 체크포인트 파일을 모델에 로드
#model.load_weights('tmp_checkpoint.ckpt')
model.load_weights('tmp_checkpoint.weights.h5')
loss, acc = model.evaluate(x_test, y_test)
print(f'체크포인트 로드 후: loss: {loss:.4f}, acc: {acc:.4f}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step - accuracy: 0.9805 - loss: 0.0797
체크포인트 로드 전: loss: 0.0797, acc: 0.9805
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step - accuracy: 0.9770 - loss: 0.0778
체크포인트 로드 후: loss: 0.0778, acc: 0.9770


## 5-2. 조기종료

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), 
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(32, activation='relu'), 
    tf.keras.layers.Dense(10, activation='softmax'), # 노드 10개로 생성
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
# EarlyStopping 콜백 생성
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [9]:
model.fit(x_train, y_train, 
          validation_data=(x_test, y_test), 
          epochs=20, 
          callbacks=[earlystopping]
          )

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9332 - loss: 0.2284 - val_accuracy: 0.9597 - val_loss: 0.1266
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9714 - loss: 0.0947 - val_accuracy: 0.9671 - val_loss: 0.1031
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9795 - loss: 0.0672 - val_accuracy: 0.9742 - val_loss: 0.0802
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9833 - loss: 0.0526 - val_accuracy: 0.9774 - val_loss: 0.0748
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9869 - loss: 0.0396 - val_accuracy: 0.9733 - val_loss: 0.0978
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9889 - loss: 0.0335 - val_accuracy: 0.9771 - val_loss: 0.0824
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9909 - loss: 0.0276 - val_accuracy: 0.9788 - val_loss: 0.0805


## 5-3. 학습률 스케줄러

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), 
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(32, activation='relu'), 
    tf.keras.layers.Dense(10, activation='softmax'), # 노드 10개로 생성
])

In [11]:
'''
def scheduler(epoch, lr):
    tf.print(f'learning_rate: {lr:.5f}')
    # 첫 5 에포크 동안 유지
    if epoch < 5:
        return lr
    else:
    # 학습률 감소 적용
        return lr * tf.math.exp(-0.1)
'''

def scheduler(epoch, lr):
    print(f'learning_rate: {lr:.5f}')  # tf.print → print 로 변경
    if epoch < 5:
        return float(lr)  # 명시적으로 float 반환
    else:
        return float(lr * tf.math.exp(-0.1))  # float으로 변환

# 콜백 객체생성 및 scheduler 함수 적용
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [12]:
model.compile(tf.keras.optimizers.SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# 초기 학습률 확인(0.01)
#print(round(model.optimizer.lr.numpy(), 5))
print(round(model.optimizer.learning_rate.numpy(), 5))

0.01


In [13]:
model.fit(x_train, y_train, 
          validation_data=(x_test, y_test), 
          epochs=10,
          # 학습률 스케줄러 적용
          callbacks=[lr_scheduler]
          )
# 최종 학습률 스케줄러 확인
#round(model.optimizer.lr.numpy(), 5)
round(model.optimizer.learning_rate.numpy(), 5)

learning_rate: 0.01000
Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8045 - loss: 0.6896 - val_accuracy: 0.9124 - val_loss: 0.2960 - learning_rate: 0.0100
learning_rate: 0.01000
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9226 - loss: 0.2678 - val_accuracy: 0.9265 - val_loss: 0.2447 - learning_rate: 0.0100
learning_rate: 0.01000
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9393 - loss: 0.2090 - val_accuracy: 0.9469 - val_loss: 0.1844 - learning_rate: 0.0100
learning_rate: 0.01000
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9501 - loss: 0.1719 - val_accuracy: 0.9531 - val_loss: 0.1606 - learning_rate: 0.0100
learning_rate: 0.01000
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9582 - loss: 0.1456 - val_accuracy: 0.9583 - val_loss: 0.1400 - learning_rate: 0.0100
learning_rate: 0.01000
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9647 - loss: 0.1250 - va

np.float32(0.00607)

## 5-4. 텐서보드

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), 
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(32, activation='relu'), 
    tf.keras.layers.Dense(10, activation='softmax'), # 노드 10개로 생성
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
# 텐서보드 저장 경로 지정
log_dir = 'tensorboard'

# 텐서보드 콜백 정의
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x_train, y_train,
            validation_data=(x_test, y_test), 
            epochs=10, 
            callbacks=[tensorboard],
            )

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9323 - loss: 0.2263 - val_accuracy: 0.9616 - val_loss: 0.1199
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9704 - loss: 0.0954 - val_accuracy: 0.9644 - val_loss: 0.1146
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9802 - loss: 0.0639 - val_accuracy: 0.9767 - val_loss: 0.0795
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9840 - loss: 0.0501 - val_accuracy: 0.9753 - val_loss: 0.0841
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9870 - loss: 0.0404 - val_accuracy: 0.9778 - val_loss: 0.0777
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9895 - loss: 0.0329 - val_accuracy: 0.9758 - val_loss: 0.0957
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9909 - loss: 0.0285 - val_accuracy: 0.9780 - val_loss: 0.0816
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9922 - loss: 0.0237 - 

In [18]:
# 텐서보드 extension 로드
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [22]:
# 텐서보드 출력 매직 커멘드
#%tensorboard --logdir {log_dir}
%tensorboard --logdir=your_log_dir